# LIBRERIAS Y CARGA DE DATOS

## LIBRERIAS

In [14]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import xgboost as xgb
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_predict
from sklearn.metrics import r2_score
from joblib import dump
from os.path import join

In [15]:
pd.set_option('display.float_format', '{:.5f}'.format)

## DATOS

In [16]:
df_casen = pd.read_csv(join("data_casen_2017", "casen_2017.csv"), sep= ",")

C:\Users\Felipe\AppData\Local\Temp\ipykernel_21864\3912985514.py:1: DtypeWarning: Columns (25,26,34,46,48,51,60,62,76,77,78,93,96,97,99,108,109,117,136,137,138,140,147,151,152,153,154,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,185,186,187,188,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,212,235,240,252,255,257,258,269,272,273,274,275,276,279,280,281,282,283,285,286,289,290,291,292,293,294,295,305,307,323,336,337,353,357,365,373,398,399,403,404,405,409,412,414,452,463,470,495,497,506,509,510,512,514,517,518,521,522,528,529,531,532,533,534,535,536,537,540,542,568) have mixed types. Specify dtype option on import or set low_memory=False.
  df_casen = pd.read_csv(join("data_casen_2017", "casen_2017.csv"), sep= ",")


Se seleccionan solo variables relevantes

In [18]:
df_casen = df_casen[["folio", "o", "nucleo", "esc", "pco1", "v4",
                     "sexo", "edad", "numper", "v6", "v27a", "o1",
                     "v30a", "zona", "v2", "v20", "ypch", 'expc',
                     'ytoth', 'ytotcorh']]

df_casen['esc'] = df_casen['esc'].apply(lambda x: min(x, 21))
df_casen['edad'] = df_casen['edad'].apply(lambda x: min(x, 100))

# VARIABLES DISCRETAS

In [19]:

# ** techo zinc
df_casen['techo_zinc'] = df_casen['v6'].isin(['Planchas metálicas (zinc, cobre, etc.)']).astype(int)

# ** techo paja
df_casen['techo_paja'] = np.where(pd.isna(df_casen['v6']), np.nan, (df_casen['v6'] == 'Paja, coirón, totora o caña').astype(int))

# ** hogar urbano
df_casen['hogar_urbano'] = np.where(pd.isna(df_casen['zona']), np.nan, (df_casen['zona'] == 'Urbano').astype(int))

# ** paredes ladrillo
df_casen['paredes_ladrillo'] = np.where(pd.isna(df_casen['v2']), np.nan, (df_casen['v2'] == 'Albañilería (bloque de cemento, piedra o ladrillo)').astype(int))

# ** paredes madera
df_casen['paredes_madera'] = df_casen['v2'].isin(['Tabique forrado por ambas caras (madera, acero, lata u otro)', 'Tabique sin forro interior (madera u otro)']).astype(int)

# ** piso radier
df_casen['piso_radier'] = np.where(pd.isna(df_casen['v4']), np.nan, (df_casen['v4'] == 'Radier').astype(int))
# ** piso madera
df_casen['piso_madera'] = np.where(pd.isna(df_casen['v4']), np.nan, (df_casen['v4'] == 'Parquet, madera, piso flotante o similar').astype(int))

# ** mujer jefe hogar
df_casen['mujer_jefe_hogar'] = df_casen['folio'].isin(df_casen[(df_casen['pco1'] == 'Jefe(a) de hogar') & (df_casen['sexo'] == 'Mujer')]['folio']).astype(int)

# ** jefe trabajo semana pasada
df_casen['jefe_trabajo_semana_pasada'] = df_casen['folio'].map(df_casen[df_casen['pco1'] == 'Jefe(a) de hogar'].set_index('folio')['o1'].eq('Sí')).fillna(False).astype(int)

# VARIABLES CONTINUAS

In [20]:

# ** educacion jefe de hogar
df_casen['educacion_jefe_hogar'] = df_casen['folio'].map(df_casen[df_casen['pco1'] == 'Jefe(a) de hogar'].set_index('folio')['esc']).values

# ** numero de personas en el hogar
df_casen['numero_personas'] = df_casen['numper']

# ** [numero de personas en el hogar]^2
df_casen['numero_personas_2'] = df_casen['numper'] ** 2

# ** [numero de personas en el hogar]^3
df_casen['numero_personas_3'] = df_casen['numper'] ** 3

# ** numero niños en en el hogar
df_casen['numero_niños'] = df_casen['folio'].map(df_casen[df_casen['edad'].between(0, 14)].groupby('folio').size()).fillna(0).astype(int)

# ** fraccion niños
df_casen['fraccion_niños'] = df_casen['numero_niños'] / df_casen['numper']

# ** numero habitaciones
df_casen['numero_habitaciones'] = df_casen['v27a']
df_casen["numero_habitaciones"].replace("No sabe/no responde", np.nan, inplace=True)
df_casen["numero_habitaciones"] = pd.to_numeric(df_casen["numero_habitaciones"], errors='coerce')
df_casen['numero_habitaciones'] = df_casen['numero_habitaciones'].apply(lambda x: min(x, 6))

# ** numero ancianos
df_casen['numero_ancianos'] = df_casen['folio'].map(df_casen[df_casen['edad'] > 65].groupby('folio').size()).fillna(0).astype(int)

# ** fraccion ancianos
df_casen['fraccion_ancianos'] = df_casen['numero_ancianos'] / df_casen['numper']

# ** edad promedio hogar
# Calcular la edad promedio por hogar
edad_promedio_hogar = df_casen.groupby('folio')['edad'].mean()

# Mapear la edad promedio al dataframe original
df_casen['edad_promedio_hogar'] = df_casen['folio'].map(edad_promedio_hogar)

# ** [edad promedio hogar]^2
df_casen['edad_promedio_hogar_2'] = df_casen['edad_promedio_hogar'] ** 2

# ** densidad hogar
df_casen['densidad_hogar'] = df_casen['numero_personas'] / df_casen['numero_habitaciones']

# ** educacion promedio hogar
df_casen['educacion_promedio_hogar'] = df_casen['folio'].map(df_casen.groupby('folio')['esc'].mean())

In [21]:
df_casen = df_casen[df_casen['numero_habitaciones'] > 0]

In [22]:
agg_methods = {
    'educacion_jefe_hogar': 'max',
    'techo_zinc': 'max',
    'techo_paja': 'max',
    'hogar_urbano': 'max',
    'paredes_ladrillo': 'max',
    'paredes_madera': 'max',
    'numero_personas': 'max',
    'numero_personas_2': 'max',
    'numero_niños': 'max',
    'fraccion_niños': 'max',
    'ypch': 'max',
    'ytoth': 'max',
    'ytotcorh': 'max',
    'mujer_jefe_hogar': 'max',
    'numero_habitaciones': 'max',
    'densidad_hogar': 'max',
    'numero_ancianos' : 'max',
    'fraccion_ancianos': 'max',
    'numero_personas_3': 'max',
    'edad_promedio_hogar': 'max',
    'edad_promedio_hogar_2': 'max',
    'educacion_promedio_hogar': 'max',
    'piso_radier': 'max',
    'piso_madera': 'max',
    'jefe_trabajo_semana_pasada': 'max'
}

df_hogares = df_casen.groupby(['folio']).agg(agg_methods).reset_index()

# MODELOS ML

In [23]:
df_hogares_regresion = df_hogares.copy()  # Se usa copy() para evitar cambios en el DataFrame original

df_hogares_regresion = df_hogares_regresion.replace([np.inf, -np.inf], np.nan)

# Se eliminan filas con valores NaN
df_hogares_regresion.dropna(inplace=True, subset= ['educacion_jefe_hogar', 'numero_habitaciones', 'densidad_hogar', 'educacion_promedio_hogar'])

df_hogares_regresion = df_hogares_regresion[df_hogares_regresion['ytotcorh'] > 0.0].copy()

X = df_hogares_regresion[['educacion_jefe_hogar', 'techo_zinc', 'techo_paja', 'hogar_urbano',
                'paredes_ladrillo', 'paredes_madera', 'numero_personas', 'numero_personas_2', 
                'numero_niños', 'fraccion_niños', 'mujer_jefe_hogar', 'numero_habitaciones',
                'densidad_hogar', 'numero_ancianos', 'fraccion_ancianos', 'numero_personas_3',
                'edad_promedio_hogar', 'edad_promedio_hogar_2', 'educacion_promedio_hogar', 
                'piso_radier', 'piso_madera', 'jefe_trabajo_semana_pasada']]


# todas las columnas en X tipo float64
X = X.astype('float64')

## OLS

In [24]:
X = sm.add_constant(X)  # se agrega la constante de OLS

# Definicion de la variable dependiente
y = df_hogares_regresion['ytotcorh']

# y tambien tipo float64
y = y.astype('float64')

y_log = np.log(y)

# Ajuste del modelo de regresion
model = sm.OLS(y_log, X).fit(cov_type='HC3')

# resumen del modelo
summary = model.summary()
print(summary)
# Guardar el modelo en un archivo
dump(model, 'modelos\OLS_2017.pkl')

                            OLS Regression Results                            
Dep. Variable:               ytotcorh   R-squared:                       0.464
Model:                            OLS   Adj. R-squared:                  0.464
Method:                 Least Squares   F-statistic:                     2586.
Date:                Mon, 11 Dec 2023   Prob (F-statistic):               0.00
Time:                        01:48:45   Log-Likelihood:                -58693.
No. Observations:               70216   AIC:                         1.174e+05
Df Residuals:                   70193   BIC:                         1.176e+05
Df Model:                          22                                         
Covariance Type:                  HC3                                         
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const               

['modelos\\OLS_2017.pkl']

## LASSO

In [25]:
# Definicion del espacio de hiperparametros para Lasso
param_grid_lasso = {
    'alpha': [1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100],
}

# Modelo Lasso
lasso_model = Lasso(max_iter=10000)

# Grid search para Lasso
grid_search_lasso = GridSearchCV(estimator=lasso_model, param_grid=param_grid_lasso, cv=5, n_jobs=-1, verbose=2)

# Ajustar el modelo utilizando el Grid Search
grid_search_lasso.fit(X, y_log)

# Mejor modelo Lasso despues de la busqueda en cuadricula
best_lasso_model = grid_search_lasso.best_estimator_

print("Mejores hiperparametros encontrados para Lasso: ", grid_search_lasso.best_params_)

# Obtener las predicciones con el mejor modelo usando validacion cruzada
y_log_pred_lasso = cross_val_predict(best_lasso_model, X, y_log, cv=5)

# R^2 para Lasso
r2_lasso = r2_score(y_log, y_log_pred_lasso)

# Ajustar el mejor modelo Lasso al conjunto de datos completo
best_lasso_model.fit(X, y_log)

print("Intercepto del mejor modelo Lasso:", best_lasso_model.intercept_)
print("\nCoeficientes del mejor modelo Lasso:")
for coef, feature in zip(best_lasso_model.coef_, X.columns):
    print(f"{feature}: {coef}")

print("\nR-cuadrado del modelo Lasso con validacion cruzada:", r2_lasso)

Fitting 5 folds for each of 7 candidates, totalling 35 fits
Mejores hiperparámetros encontrados para Lasso:  {'alpha': 0.0001}
Intercepto del mejor modelo Lasso: 10.670586875927949

Coeficientes del mejor modelo Lasso:
const: 0.0
educacion_jefe_hogar: 0.00612453898786581
techo_zinc: -0.11853035960303704
techo_paja: -0.0
hogar_urbano: 0.07922584380290962
paredes_ladrillo: -0.10207953578789933
paredes_madera: -0.16948340471794038
numero_personas: 0.464614865927765
numero_personas_2: -0.04758018302994231
numero_niños: -0.0055334027224660165
fraccion_niños: -0.16669207574505002
mujer_jefe_hogar: -0.05388173192614696
numero_habitaciones: 0.05219295712799875
densidad_hogar: -0.05631806671297088
numero_ancianos: 0.04598219049958878
fraccion_ancianos: 0.06465437030788472
numero_personas_3: 0.0019545527906276645
edad_promedio_hogar: 0.025866519777301103
edad_promedio_hogar_2: -0.00016047010317535006
educacion_promedio_hogar: 0.08524307982326973
piso_radier: -0.05118434313300953
piso_madera: 0.0

## RANDOM FOREST

In [26]:
X = X.drop('const', axis=1)  # Sacamos la constante de OLS

In [27]:
# Definicion de la cuadricula de parametros para la busqueda
param_distributions = {
    'n_estimators': [10, 50, 100, 200],
    'max_features': ['auto', 'sqrt'],
    'max_depth': [10, 20, 30, 40, 50, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Random Forest con un estado aleatorio para reproducibilidad
rf_model = RandomForestRegressor(random_state=42, oob_score=True)

rf_model.fit(X, y_log)

# Cálculo de errores OOB y desviacion estandar
oob_errors = abs(y_log - rf_model.oob_prediction_)
error_std = np.std(oob_errors)

# Busqueda aleatoria con validacion cruzada
random_search = RandomizedSearchCV(estimator=rf_model, param_distributions=param_distributions, n_iter=100, cv=5, n_jobs=-1, verbose=2, random_state=42)
random_search.fit(X, y_log)

# Mejor modelo Random Forest despues de RandomizedSearchCV
best_rf_model = random_search.best_estimator_

# Prediccion con el mejor modelo
y_log_pred_rf = best_rf_model.predict(X)

# Cálculo de R^2
r2_rf = r2_score(y_log, y_log_pred_rf)

# Importancia de las caracteristicas
feature_importances = best_rf_model.feature_importances_

print("Mejores hiperparametros encontrados: ", random_search.best_params_)

print("\nImportancia de las caracteristicas:")
for importance, feature in sorted(zip(feature_importances, X.columns), reverse=True):
    print(f"{feature}: {importance}")

print("\nR-cuadrado del modelo Random Forest con el mejor modelo:", r2_rf)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Mejores hiperparámetros encontrados:  {'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 40}

Importancia de las características:
educacion_promedio_hogar: 0.2390129459931674
educacion_jefe_hogar: 0.1761334342706579
edad_promedio_hogar_2: 0.06986687842775953
edad_promedio_hogar: 0.06782681992623202
numero_personas_3: 0.056640277464367395
numero_personas: 0.05587365568346177
jefe_trabajo_semana_pasada: 0.05315815123986081
numero_habitaciones: 0.05309068161145577
numero_personas_2: 0.05238668033054868
densidad_hogar: 0.02877284003855752
techo_zinc: 0.025981574805629848
paredes_madera: 0.021336656087452544
fraccion_niños: 0.016708849905994923
mujer_jefe_hogar: 0.015747069101395433
hogar_urbano: 0.015547828513977081
piso_madera: 0.015343340086266338
fraccion_ancianos: 0.009898305594285934
paredes_ladrillo: 0.008723678963744338
numero_niños: 0.007930892213060658
numero_anci

In [28]:
dump(error_std, 'modelos/error_std.joblib')
dump(best_rf_model, 'modelos/modelo_rf_oob_2017.joblib')

['modelos/modelo_rf_oob_2017.joblib']

## XGBOOST

In [29]:

# Definicion del espacio de hiperparametros para XGBoost
param_distributions_xgb = {
    'n_estimators': [50, 100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 4, 5, 6, 7],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

# Definicion del modelo inicial
xgb_model = xgb.XGBRegressor(objective='reg:squarederror')

# Busqueda aleatoria para XGBoost
random_search_xgb = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_distributions_xgb, n_iter=100, cv=5, n_jobs=-1, verbose=2, random_state=42)
random_search_xgb.fit(X, y_log)

# Mejor modelo XGBoost despues de RandomizedSearchCV
best_xgb_model = random_search_xgb.best_estimator_

print("Mejores hiperparametros encontrados para XGBoost: ", random_search_xgb.best_params_)

# Obtener las predicciones con el mejor modelo usando validacion cruzada
y_log_pred_xgb = cross_val_predict(best_xgb_model, X, y_log, cv=5)

# R^2 para XGBoost
r2_xgb = r2_score(y_log, y_log_pred_xgb)

# Ajustar el mejor modelo al conjunto de datos completo
best_xgb_model.fit(X, y_log)
feature_importances_xgb = best_xgb_model.feature_importances_

print("Pesos de las caracteristicas para XGBoost:")
for importance, feature in sorted(zip(feature_importances_xgb, X.columns), reverse=True):
    print(f"{feature}: {importance}")

print("\nR-cuadrado del modelo XGBoost con validacion cruzada:", r2_xgb)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Mejores hiperparametros encontrados para XGBoost:  {'subsample': 0.9, 'n_estimators': 300, 'max_depth': 4, 'learning_rate': 0.1, 'colsample_bytree': 0.9}
Pesos de las caracteristicas para XGBoost:
numero_personas_3: 0.25294747948646545
numero_personas_2: 0.14604170620441437
educacion_promedio_hogar: 0.11265992373228073
numero_personas: 0.09400477260351181
jefe_trabajo_semana_pasada: 0.06718194484710693
numero_habitaciones: 0.04496609792113304
techo_zinc: 0.04089939966797829
educacion_jefe_hogar: 0.03942055255174637
paredes_madera: 0.03761685639619827
numero_ancianos: 0.02999485470354557
mujer_jefe_hogar: 0.026158291846513748
hogar_urbano: 0.02570302039384842
fraccion_niños: 0.018939871340990067
piso_madera: 0.014506948180496693
edad_promedio_hogar: 0.013406612910330296
edad_promedio_hogar_2: 0.008789343759417534
fraccion_ancianos: 0.00798605103045702
piso_radier: 0.006841307505965233
densidad_hogar: 0.006065054796636105
par